Change log V5:
* some of tracking have multiple split, which will affect long splits
* adding column *split_sum_check* to flag them
* end counting with another split

**last edit: 16.3.2022**

Change log V4a:
* add tag for short split with and without merge
* add tag for long split with and without merge

**last edit: 10.3.2022**

Change log V4:
* min track length problem
* count/add long splits [TODO: check results]

**last edit: 28.2.2022**

Change log V3:
* do not count short splits that are not merged back
* add number of splits to output, based on information from tracks
* add table output, spots count for track based on frame/time

**last edit: 19.1.2022**

https://forum.image.sc/t/trackmate-parse-exact-relationships-of-splitting-events-within-each-track/30487/6

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

#@markdown Path to directory (needs to be specified by user)
datadir = "/content/gdrive/MyDrive/Python/tracking_analysis/data/set1/"  #@param {type: "string"}
#@markdown ---
#@markdown Default Trackmate file names
tracks_name = "tracks_table.csv" #@param {type: "string"}
edges_name = "edges_table.csv" #@param {type: "string"}
spots_name = "spots_table.csv" #@param {type: "string"}

tracks = pd.read_csv(datadir+tracks_name,\
                     usecols = ['NUMBER_SPOTS', 'NUMBER_GAPS', 'LONGEST_GAP', 'NUMBER_SPLITS', 'TRACK_ID'])
links = pd.read_csv(datadir+edges_name, \
                    usecols = ['LABEL', 'TRACK_ID', 'SPOT_SOURCE_ID', 'SPOT_TARGET_ID', 'EDGE_X_LOCATION', \
                               'EDGE_Y_LOCATION'])
spots = pd.read_csv(datadir+spots_name,\
                    usecols = ['LABEL', 'ID', 'TRACK_ID', 'POSITION_X', 'POSITION_Y', 'FRAME'])




/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,2,4,5,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# tracks.head()

In [ ]:
tracks.describe()

,TRACK_ID,NUMBER_SPOTS,NUMBER_GAPS,NUMBER_SPLITS,LONGEST_GAP
count,356,356,356,356,356
unique,356,153,20,22,7
top,Track ID,220,0,0,3
freq,1,23,78,270,86


In [ ]:
#@title
# links.head()


Generate a list of spot_ids that correspond to a splitting event

In [ ]:
#@title
# Generate a list of spot_ids that correspond to a splitting event
# (SOURCE_IDs of splitting event appear twice)

source_ids = list(links["SPOT_SOURCE_ID"])
source_id_counts = Counter(source_ids)
splitting_event_ids = [id for id in source_id_counts if source_id_counts[id] > 1]

merge_ids = list(links["SPOT_TARGET_ID"])
merge_id_counts = Counter(merge_ids)
merging_event_ids = [id for id in merge_id_counts if merge_id_counts[id] > 1]

# Add Boolean to Spots and Links Dataframes

spots["Splitting_event"] = spots["ID"].apply(lambda x:\
                                             False if x not in splitting_event_ids\
                                             else True)

links["Splitting_event"] = links["SPOT_SOURCE_ID"].apply(lambda x:\
                                             False if x not in splitting_event_ids\
                                             else True)

links["Merging_event"] = links["SPOT_TARGET_ID"].apply(lambda x:\
                                             False if x not in merging_event_ids\
                                             else True)
# Rename link dataframe columns

links.columns = ['LABEL',\
                 'TRACK_ID',\
                 'SOURCE_ID',\
                 'TARGET_ID',\
                 'EDGE_X_LOCATION',\
                 'EDGE_Y_LOCATION',\
                 'SPLITTING_EVENT',\
                 "MERGING_EVENT"]




In [ ]:
spots.head()

,LABEL,ID,TRACK_ID,POSITION_X,POSITION_Y,FRAME,Splitting_event
0,Label,Spot ID,Track ID,X,Y,Frame,False
1,Label,Spot ID,Track ID,X,Y,Frame,False
2,NaN,NaN,NaN,(micron),(micron),NaN,False
3,ID195074,195074,0,72.26007944548479,29.066845262534855,0,False
4,ID193538,193538,0,71.30581333818493,29.213162111215123,7,True


In [ ]:
links.head()

,LABEL,TRACK_ID,SOURCE_ID,TARGET_ID,EDGE_X_LOCATION,EDGE_Y_LOCATION,SPLITTING_EVENT,MERGING_EVENT
0,Label,Track ID,Source spot ID,Target spot ID,Edge X,Edge Y,False,False
1,Label,Track ID,Source ID,Target ID,Edge X,Edge Y,False,False
2,NaN,NaN,NaN,NaN,(micron),(micron),False,False
3,ID222772 ? ID222959,0,222772,222959,72.18021229623858,29.4176709673943,False,False
4,ID194684 ? ID193917,0,194684,193917,72.61823009423327,28.15439177150136,False,False


Create a time-spreadsheat for visual inspection of number of spots in each track

In [ ]:
#@title
# retype to int, from 4th
number_column = spots.loc[3::,'FRAME'].astype(int)
maxFrame=number_column.max()
print("Number of time frames",maxFrame)

d = {'1': np.zeros(maxFrame)}
numTracks = pd.DataFrame(data=d)


N=tracks.iloc[3::]['TRACK_ID'].count()
#print(N)
for id in range(0,N):
  numTracks.insert(id, tracks.iloc[3+id]['TRACK_ID'], np.zeros(maxFrame), True)
  #print(tracks.iloc[3+id]['TRACK_ID'])

Number of time frames 219


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if self.run_code(code, result):


add merging events to links

In [ ]:
#@title
from pandas.core.dtypes.cast import maybe_infer_to_datetimelike
for id in range(0,N):
  # go through all tracks
  id_num=tracks.iloc[3+id]['TRACK_ID']

  #select spots only for one id
  spots_ID=spots[spots['TRACK_ID']==id_num]
  links_ID=links[links['TRACK_ID']==id_num]

  #print(minFrame, maxFrame)
  # inspect only tracks with splits
  TC=len(spots_ID)
  if TC>0:
    #print(TC)
    minFrame=int(spots_ID['FRAME'].min())
    maxFrame=int(spots_ID['FRAME'].max())
    for fr in range(minFrame,maxFrame):
      nFrames=spots_ID[spots_ID['FRAME']==str(fr)]
      numTracks.at[fr,id_num]=len(nFrames)
      #print(fr, len(nFrames))

In [ ]:
numTracks.head()

,0,2,3,4,5,6,7,8,9,11,...,710,711,713,715,729,731,733,734,736,1
0,2.0,4.0,4.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,5.0,4.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,5.0,4.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.0,5.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.0,4.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Look for splits and merges in each track

In [ ]:
#@title int(c), merge, split = findLinks(start_ID, links_ID, splitCut) 
def findLinks(start_ID, links_ID, splitCut) -> int:
  c=1
  merge=False
  split=False
  # find link which has first spot id as source
  link=links_ID[links_ID['SOURCE_ID']==start_ID]

  if len(link)>0:
    # find where it leads
    start_ID=link['TARGET_ID'].values[0]

    # repeat until merge or end is found
    while len(link)>0: #(link['MERGING_EVENT'].values[0]==False) & len(link)>0:
      # check for merge
      if (link['MERGING_EVENT'].values[0]==True):
        merge=True
        break
      #  check for split, but after merge - megre has prioritz in counting steps
      if splitCut:
        if (link['SPLITTING_EVENT'].values[0]==True):
          print('Spliting again at', start_ID)
          split=True
          break
      # find link which has first spot id as source
      try:
        link=links_ID[links_ID['SOURCE_ID']==start_ID]
        if len(link)==0:
          break
      except:
        break
      
      # find where it leads
      start_ID=link['TARGET_ID'].values[0]
      c=c+1;
    return int(c), merge, split
  else:
    return 0, merge, split

In [ ]:
#@title unique function
# function to get unique values
def unique(list1):
 
    # initialize a null list
    unique_list = []
     
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return unique_list

* short_split: <th without merge
* short_split_merge: <th and merge
* long_split: >th without merge
* long_split_merge: >th and merge

In [ ]:
#@title
tracks['short_split'] = 0
tracks['short_split_merge'] = 0
tracks['num_splits'] = 0
tracks['num_merges'] = 0
tracks['long_split'] = 0
tracks['long_split_merge'] = 0
tracks['split_sum_check'] = 0
#tracks['short_split'].values[0]='Number of short splits'
#tracks['short_split'].values[1]='shorter then 2 frames'
#tracks['long_split'].values[0]='Number of long splits'
#tracks['long_split'].values[1]='longer then 2 frames'

#@markdown Definu number of frames in from split to megre to be defined as short split
short_split_length=3 #@param {type: "slider", min: 2, max: 20}
#@markdown End counting with another split event
splitCut = False  #@param {type: "boolean"}


#for id in range(0,N):
for id in range(0,N):
  # go through all tracks
  id_num=tracks.iloc[3+id]['TRACK_ID']
  print('--------------------')
  print('Track ID', id_num)

  #select spots only for one id
  spots_ID=spots[spots['TRACK_ID']==id_num]
  # mask = mask = spots_ID['FRAME'].isin(['6','5','4','3','2','1'])
  # spots_ID=spots_ID[mask]

  links_ID=links[links['TRACK_ID']==id_num]


  # select spliting links only
  links_ID_split=links_ID[links_ID['SPLITTING_EVENT']==True]
  print('num links: ',len(links_ID_split))
  num=[]
  mergeT=[]
  splitT=[]

  # check if there is any split
  if len(links_ID_split)>0:
    # inspect every spliting link
    for Tid in range(0,len(links_ID_split)):
      start_ID=links_ID_split['TARGET_ID'].values[Tid]
      # check if it exists
      if len(start_ID)>1: 
        numLinks, mergeTag, splitTag = findLinks(start_ID, links_ID,splitCut)
        num.append(numLinks)
        mergeT.append(mergeTag)
        splitT.append(splitTag)

    print('Start ID: ',start_ID, ', tracing: ', num) #, ' merge', mergeTag)
    print(mergeT)
    print(splitT)

    #shortS = [n for n in num if n <=short_split_length]
    shortS = [n for i,n in enumerate(num) if n <=short_split_length and not mergeT[i] and not splitT[i]]
    shortSmerge = [n for i,n in enumerate(num) if n <=short_split_length and mergeT[i] and not splitT[i]]
    shortSsplit = [n for i,n in enumerate(num) if n <=short_split_length and splitT[i]]
    #longS = [n for n in num if n >short_split_length]
    longS = [n for i,n in enumerate(num) if n >short_split_length and not mergeT[i] and not splitT[i]]
    longSmerge = [n for i,n in enumerate(num) if n >short_split_length and mergeT[i] and not splitT[i]]
    longSsplit = [n for i,n in enumerate(num) if n >short_split_length and splitT[i]]
    
    # we will have two times of short splits, since we end counting at merge
    tracks['short_split'].values[3+id]=len(shortS)#max(len(shortS)/2-len(shortSmerge)/2,0)
    tracks['short_split_merge'].values[3+id] = len(shortSmerge)/2 #- len(shortSsplit)

    #long split (to be checked)
    tracks['long_split'].values[3+id]=len(longS)#max(len(longS)/2-(len(shortS)/2)-len(longSmerge)/2,0)
    tracks['long_split_merge'].values[3+id] = len(longSmerge)/2 #- len(longSsplit)

    split=links_ID[links_ID['SPLITTING_EVENT']==True]
    links_splitU=split["SOURCE_ID"].tolist()
    # number of splits
    tracks['num_splits'].values[3+id]=len(unique(links_splitU))

    merge=links_ID[links_ID['MERGING_EVENT']==True]
    links_mergedU=merge["TARGET_ID"].tolist()
    # number of merges
    tracks['num_merges'].values[3+id]=len(unique(links_mergedU))

    tracks['split_sum_check'].values[3+id]=tracks['num_splits'].values[3+id]-tracks['short_split'].values[3+id]-tracks['short_split_merge'].values[3+id]-tracks['long_split'].values[3+id]-tracks['long_split_merge'].values[3+id]

--------------------
Track ID 0
num links:  8
Start ID:  212677 , tracing:  [2, 1, 2, 38, 2, 1, 2, 2]
[False, True, True, True, True, True, True, True]
[False, False, False, False, False, False, False, False]
--------------------
Track ID 2
num links:  54
Start ID:  207700 , tracing:  [3, 8, 2, 1, 11, 4, 7, 2, 2, 4, 12, 4, 10, 4, 9, 4, 1, 1, 4, 15, 3, 1, 4, 2, 3, 38, 13, 1, 1, 38, 38, 3, 1, 1, 45, 58, 2, 34, 1, 32, 4, 2, 27, 26, 4, 43, 1, 7, 1, 1, 2, 4, 1, 46]
[True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, Fal

In [ ]:
#@title
# if len(links_ID_split)>0:
#     # inspect every spliting link
#     for Tid in range(0,len(links_ID_split)):
#       start_ID=links_ID_split['TARGET_ID'].values[Tid]
#       # check if it exists
#       if len(start_ID)>1: 
#         numLinks, mergeTag = findLinks(start_ID, links_ID)
#         print("Steps found: ", numLinks)
#         print("Tag:", mergeTag)

## Inspect results

In [ ]:
tracks[tracks["TRACK_ID"]=='2']

,TRACK_ID,NUMBER_SPOTS,NUMBER_GAPS,NUMBER_SPLITS,LONGEST_GAP,short_split,short_split_merge,num_splits,num_merges,long_split,long_split_merge,split_sum_check
4,2,904,12,21,2,2,11,27,28,1,14,-1


In [ ]:
#@title
tracks

In [ ]:
#@title
# #@title
# sel=tracks['TRACK_ID']=='40'
# #tracks[sel]
# tracks[sel]

## Export result to csv

In [ ]:
output_name = "tracks_output.xlsx" #@param {type: "string"}
tracks.to_excel(output_name,
             sheet_name='Tracks_analysis')  

In [ ]:
frames_output_name = "timeFrames_output.xlsx" #@param {type: "string"}
numTracks.to_excel(frames_output_name,
             sheet_name='Tracks_analysis')  